![title](/home/francescocremonesiext/new-areas/imaging-directions.png)

In [ ]:
import numpy as np
import nibabel as nib
from skimage.morphology import binary_dilation, binary_erosion, label
import os
import matplotlib.pyplot as plt
%matplotlib inline
from typing import List, Dict
from functools import reduce
from tqdm import tqdm

: 

## Generic functions

This section defines the functions needed for rule-based segmentation of aires ganglionnaires.

In [ ]:
def get_extremal_idx(organ_segmentation: 'ImageSegmentation', axis: int, get_largest_index: bool = True) -> int:
    """Gets the index corresponding to the first or last nonzero pixel on an axis.
    
    
    Args:
        organ_segmentation: the binary mask representing the segmentation of an organ
        axis: the integer corresponding to the axis where we want to find the extreme point (e.g. 2 for z axis)
        get_largest_index: boolean to get the largest or the smallest index on the projection

    Examples:
        for the z axis, the position of the highest pixel in the organ segmentation can be found with
        ```
        get_extremal_idx(organ_segmentation, 2, get_largest_index=True)
        ```

        while the following finds the position of the lowest pixel in the organ segmentation
        ```
        get_extremal_idx(organ_segmentation, 2, get_largest_index=False)
        ```

    Returns:
        pixel_index: the integer index corresponding to the last (resp. first) nonzero pixel on the axis

    """
    all_ax = [0,1,2]
    all_ax.pop(axis)
    projection = organ_segmentation
    for ax_ in sorted(all_ax)[::-1]:
        projection = projection.sum(ax_)
    pixel_index = np.max(np.where(projection > 0)[0]) if get_largest_index else np.min(np.where(projection > 0)[0])
    return pixel_index

In [ ]:
def define_area_by_plane(organ_segmentation: 'ImageSegmentation', axis: int, get_largest_index: bool = True, one_after: bool = True) -> 'ImageSegmentation':
    """Returns a new mask that defines a 3d-rectangular area in the image.

    The new mask is split in two areas, divided by the plane perpendicular to `axis` at the position 
    defined by `get_extremal_idx(mask, axis, get_largest_index)`.
    
    If `one_after` is True, all pixels with a larger index than the plane are set to 1, all the
    others are set to 0. If `one_after` is False, the opposite happens.
    
    Args:
        organ_segmentation: the binary mask representing the segmentation of an organ
        axis: the integer corresponding to the axis where we want to find the extreme point (e.g. 2 for z axis)
        get_largest_index: boolean to get the largest or the smallest index on the projection
        one_after.

    Returns:
        area_mask: a binary image with the same shape as mask 

    """
    idx = get_extremal_idx(organ_segmentation, axis, get_largest_index)
    area_mask = np.zeros_like(organ_segmentation)
    selecting_slice = slice(idx, None) if one_after else slice(None, idx)
    match axis:
        case 0:
            indices = (selecting_slice, Ellipsis)
        case 1:
            indices = (slice(None), selecting_slice, slice(None))
        case 2:
            indices = (Ellipsis, selecting_slice)
    area_mask[indices] = 1
    return area_mask
         

In [ ]:
specs_to_args = {
    'inferior border': {'axis': 2, 'get_largest_index': False,  'one_after': True},
    'superior border': {'axis': 2, 'get_largest_index': True,  'one_after': False},
    'anterior border': {'axis': 1, 'get_largest_index': True,  'one_after': False},
    'posterior border': {'axis': 1, 'get_largest_index': False,  'one_after': True},
    'left border': {'axis': 0, 'get_largest_index': True,  'one_after': False},
    'right border': {'axis': 0, 'get_largest_index': False,  'one_after': True},
}

In [ ]:
path_to_totalseg_segmentations = '/mnt/nas/database_CAL_immuno_lucie/pet_metrics_project/runs/autopet_control_patients/seg_and_metrics_run_2024_11_22/output/SEG'
path_to_ct = '/mnt/nas/autoPET/data/FDG-PET-CT-Lesions_nifti_neg'


totalseg_tasks = ['total', 'tissue', 'head_muscles', 'head_glands_cavities', 'headneck_bones_vessels', 'headneck_muscles']

totalseg_structure_to_task = {struct: task for task in totalseg_tasks for struct in os.listdir(os.path.join(path_to_totalseg_segmentations, task, 'PETCT_1bdefef7d5_20060114'))}
totalseg_structures = list(totalseg_structure_to_task.keys())

In [ ]:
def define_area_by_specs(area_specs: Dict[str, List[Dict]], patient: str) -> 'ImageSegmentation':
    """Returns a new mask that defines an aire ganglionnaire based on a dictionary of specifications.

    The specifications explain how each border of the aire ganglionnaire is defined. 
    The format of the specs is:
    ```
    {border of the aire ganglionnaire: [structure_specs]}
    ```
    Each `structure_spec` is itself a dictionary with the keys:
    - structure: totalsegmentator filename
    - border: which border of the totalsegmentator class defines the border of the aire ganglionnaire

    Example:
    ```
    level_iia_left_specs = {
        'inferior border': [{'border': ['inferior border'], 'structure': 'hyoid'}],
        'superior border': [{'border': ['superior border'], 'structure': 'vertebrae_C1'}],
        'posterior border': [{'border': ['posterior border'], 'structure': 'internal_jugular_vein_left'}],
        'anterior border': [{'border': ['posterior border'], 'structure': 'submandibular_gland_left'}],
        'left border': [{'border': ['left border'], 'structure': 'sternocleidomastoid_left'}],
        'right border': [{'border': ['right border'], 'structure': 'internal_carotid_artery_left'}],
    }
```
"""
    def _gen_areas():
        for border, specs in area_specs.items():
            one_after_ = specs_to_args[border]['one_after']
            axis_ = specs_to_args[border]['axis']
            for organ_border_specs in specs:
                seg_filename = organ_border_specs['structure'] + '.nii.gz'
                for organ_border_name in organ_border_specs['border']:
                    get_largest_index_ = specs_to_args[organ_border_name]['get_largest_index']
                    seg_file_path = os.path.join(
                        path_to_totalseg_segmentations,
                        totalseg_structure_to_task[seg_filename],
                        patient,
                        seg_filename
                    )
                    organ_segmentation = nib.load(seg_file_path).get_fdata()
                yield define_area_by_plane(organ_segmentation, axis_,get_largest_index_, one_after_)
    return reduce(lambda x,y: x*y, _gen_areas())



## Example

In [ ]:
os.listdir(path_to_totalseg_segmentations)

In [ ]:
patient_id = 'PETCT_1bdefef7d5'
patient_id_with_date = 'PETCT_1bdefef7d5_20060114'

In [ ]:
ct_img = nib.load(os.path.join(path_to_ct, patient_id, '01-14-2006-NA-PET-CT Ganzkoerper  primaer mit KM-32502/CT.nii.gz'))
ct = ct_img.get_fdata()

In [ ]:
hyoid = nib.load(os.path.join(path_to_totalseg_segmentations, 'headneck_bones_vessels', patient_id_with_date, 'hyoid.nii.gz')).get_fdata()

Combine all totalsegmentator masks in order to remove them from the aire ganglionnaire later

In [ ]:
combined = np.zeros_like(ct, dtype=np.int32)
for structure, task in tqdm(totalseg_structure_to_task.items()):
    if 'subcutaneous_fat' in structure:
        print(f'Skipping {structure}')
        continue
    segdata = nib.load(os.path.join(
                        path_to_totalseg_segmentations,
                        task,
                        patient_id_with_date,
                        structure
                    )).get_fdata()
    combined = np.clip(combined + segdata.astype(np.int32), 0, 1)

Stupid heuristic to identify background, can be improved in the future

In [ ]:
ct_foreground = (ct > -600).astype(np.int32)

## Rule-based segmentation of aires ganglionnaires

### Level Ia - left

In [ ]:
level_ia_left_specs = {
    'inferior border': [{'border': ['inferior border'], 'structure': 'hyoid'}],
    'superior border': [{'border': ['inferior border'], 'structure': 'skull'}],  # skull is totalsegmentator equivalent for jaw
    'anterior border': [{'border': ['anterior border'], 'structure': 'digastric_left'}],
    'posterior border': [{'border': ['posterior border'], 'structure': 'submandibular_gland_left'}],
    'left border': [{'border': ['right border'], 'structure': 'digastric_left'}],
    #'medial border': {??}
}

In [ ]:
level_ia_left = define_area_by_specs(level_ia_left_specs, patient_id_with_date)

In [ ]:
level_ia_left.max()

the maximum is 0, meaning that no area was found. This is likely due to the position of the patient which puts the hyoid and jaw in the wrong place

### Level IIa - left

In [ ]:
level_iia_left_specs = {
    'inferior border': [{'border': ['inferior border'], 'structure': 'hyoid'}],
    'superior border': [{'border': ['superior border'], 'structure': 'vertebrae_C1'}],
    'posterior border': [{'border': ['posterior border'], 'structure': 'internal_jugular_vein_left'}],
    'anterior border': [{'border': ['posterior border'], 'structure': 'submandibular_gland_left'}],
    'left border': [{'border': ['left border'], 'structure': 'sternocleidomastoid_left'}],
    'right border': [{'border': ['right border'], 'structure': 'internal_carotid_artery_left'}],
}

In [ ]:
level_iia_left = define_area_by_specs(level_iia_left_specs, patient_id_with_date)

In [ ]:
# Remove all other segmented structures
level_iia_left *= 1 - combined

Some heuristics to improve 

In [ ]:
# Remove background -- ignored for now
#level_iia_left *= ct_foreground

In [ ]:
def extract_largest_connected_component(mask):
    label_img  = label(mask)
    max_vol = 0
    max_label = -1
    for lab_ in range(1,label_img.max()+1):
        vol_ = (label_img == lab_).sum()
        if vol_ > max_vol:
            max_label = lab_
            max_vol = vol_
    return (label_img == max_label).astype(np.int32)

In [ ]:
# Extract only the largest component - optionally perform erosion before and dilation after --  ignored for noe
#level_iia_left = binary_erosion(level_iia_left)   # heuristic 3 iterations
#level_iia_left = extract_largest_connected_component(level_iia_left)
#level_iia_left = binary_dilation(level_iia_left)

In [ ]:
ni_img = nib.Nifti1Image(level_iia_left, ct_img.affine, ct_img.header)
nib.save(ni_img, f'/home/francescocremonesiext/new-areas/aires-ganglionnaires/output/autopet_control_patients/{patient_id_with_date}/level-iia-left.nii.gz')

## Detailed examples

### Extremal indices

In [ ]:
# inferior border hyoid
inf_hyoid_index = get_extremal_idx(hyoid, 2, get_largest_index=False)
inf_hyoid_index

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

ax.imshow(ct[256,150:450,200:450].T, cmap='bone', origin='lower')
ax.plot([0, 300], [inf_hyoid_index-200,inf_hyoid_index-200], color='red')
ax.axis('off')

### Define area by plane

In [ ]:
# from inferior border hyoid towards the top 
area = define_area_by_plane(hyoid, 2, get_largest_index=False, one_after=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

ax.imshow(ct[256,150:450,200:450].T, cmap='bone', origin='lower')
ax.plot([0, 300], [inf_hyoid_index-200,inf_hyoid_index-200], color='red')
img  = ax.imshow(area[256,150:450,200:450].T, cmap='seismic', origin='lower', alpha=0.5)
ax.axis('off')
fig.colorbar(img)

In [ ]:
list(filter(lambda x: 'jugular' in x, totalseg_structure_to_task.keys()))

In [ ]:
#os.makedirs(f'/home/francescocremonesiext/new-areas/aires-ganglionnaires/output/autopet_control_patients/{patient_id_with_date}')